In [78]:
import pandas as pd
df_regions = pd.read_csv("Data/noc_regions.csv")
df_events = pd.read_csv("Data/athlete_events.csv")

In [79]:
df_regions.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [80]:
df_events.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## A) 
Hur många länder som är med i datan?

In [81]:
number_od_countries = df_regions["NOC"].nunique()
print(f"Det finns {number_od_countries} olika länder i datan.")

Det finns 230 olika länder i datan.


## B)
Vilka länder är med? (förkortningarna räcker)

In [82]:
countries = df_regions["NOC"]
countries

0      AFG
1      AHO
2      ALB
3      ALG
4      AND
      ... 
225    YEM
226    YMD
227    YUG
228    ZAM
229    ZIM
Name: NOC, Length: 230, dtype: object

## C) 
Vilka sporter är med?

In [83]:
df_events["Sport"]

0            Basketball
1                  Judo
2              Football
3            Tug-Of-War
4         Speed Skating
              ...      
271111             Luge
271112      Ski Jumping
271113      Ski Jumping
271114        Bobsleigh
271115        Bobsleigh
Name: Sport, Length: 271116, dtype: object

## D)
Vilka medaljtyper finns det?

In [84]:
unique_medals = df_events['Medal'].dropna().unique()
medal_string = ', '.join(unique_medals[:-1]) + f" och {unique_medals[-1]}"
print(f"Medaljtyperna är: {medal_string}.")

Medaljtyperna är: Gold, Bronze och Silver.


## E)
Ta reda på statistik för åldern: medelvärde, median, min, max, standardavvikelse

In [85]:
df_events['Age'].agg(
    Medelvärde='mean', 
    Median='median', 
    Minimum='min', 
    Maximum='max', 
    Standardavvikelse='std'
)

Medelvärde           25.556898
Median               24.000000
Minimum              10.000000
Maximum              97.000000
Standardavvikelse     6.393561
Name: Age, dtype: float64

## F)
Utforska datan vidare med egna frågor
- Vilka åldersintervaller tar flest medaljer?
- Varierar medelåldern mellan mästerskapen? 
- Könsfördelning mellan mästerskapen?

In [86]:
df_events['Medal'].fillna('None', inplace=True)

In [87]:
def age_interval(age):
    if age < 18:
        return 'a. < 18'
    elif age >= 18 and age < 25:
        return 'b. 18 - 24'
    elif age >= 25 and age < 30:
        return 'c. 25 - 29'
    elif age >= 30 and age < 40:
        return 'd. 30 - 39'
    elif age >= 40:
        return 'e. 40 +'

def medal_grp(medal):
    return 0 if medal == 'None' else 1

df_events['Age_interval'] = df_events['Age'].apply(age_interval)
df_events['Medal_grp'] = df_events['Medal'].apply(medal_grp)
medals_age = df_events.groupby('Age_interval')['Medal_grp'].mean()

print('Andel medaljvinnare per åldersgrupp:')
print(medals_age)

Andel medaljvinnare per åldersgrupp:
Age_interval
a. < 18       0.094260
b. 18 - 24    0.141284
c. 25 - 29    0.164916
d. 30 - 39    0.160950
e. 40 +       0.137807
Name: Medal_grp, dtype: float64


Den åldersgrupp med högst andel medaljer per tävlande är 25-29 med 16,5%. Åldersgrupp 30-39 kommer strax därefter med 16,1%.

In [100]:
df_events['championship'] = df_events['Year'].apply(str) + '_' + df_events['City'] + '_' + df_events['Season']

championship_age = df_events.groupby('championship')['Age'].mean()

print('Medelålder av tävlande per mästerskap:')
print(championship_age.sample(10))
print(f'\nStandradavvikelsen för medelåldern: {championship_age.std():.2f}')

Medelålder av tävlande per mästerskap:
championship
1956_Stockholm_Summer      34.445736
1980_Moskva_Summer         23.704354
1952_Oslo_Winter           25.446691
1908_London_Summer         26.970228
1968_Mexico City_Summer    24.255154
2010_Vancouver_Winter      26.124262
1992_Barcelona_Summer      24.396165
1976_Montreal_Summer       23.863837
2008_Beijing_Summer        25.734118
1948_London_Summer         29.304796
Name: Age, dtype: float64

Standradavvikelsen för medelåldern: 2.28


Ovan syns ett slumpat urval av mästerskap och deras medelålder. Medelåldern för alla mästerskap varierar med en standardavvikelse på 2,3.

In [89]:
df_events = pd.get_dummies(df_events, columns=['Sex'])
championship_sex = df_events.groupby('championship')[['Sex_M', 'Sex_F']].sum()

print('Antal tävlande män och kvinnor per mästerskap:')
print(championship_sex)

Antal tävlande män och kvinnor per mästerskap:
                                    Sex_M  Sex_F
championship                                    
1896_Athina_Summer                    380      0
1900_Paris_Summer                    1903     33
1904_St. Louis_Summer                1285     16
1906_Athina_Summer                   1722     11
1908_London_Summer                   3054     47
1912_Stockholm_Summer                3953     87
1920_Antwerpen_Summer                4158    134
1924_Chamonix_Winter                  443     17
1924_Paris_Summer                    4989    244
1928_Amsterdam_Summer                4588    404
1928_Sankt Moritz_Winter              549     33
1932_Lake Placid_Winter               330     22
1932_Los Angeles_Summer              2622    347
1936_Berlin_Summer                   6038    468
1936_Garmisch-Partenkirchen_Winter    814     81
1948_London_Summer                   5777    628
1948_Sankt Moritz_Winter              942    133
1952_Helsinki_Summer  

Fördelningen av män och kvinnor är mycket mer jämn för de senare mästerskapen än de i början. I början var andelen kvinnor några enstaka procent av alla tävlande.

## G)
Diagram över könsfördelningen

## H)
Diagram över topp 10 länder som tagit flest medaljer

## I) 
Plotta gärna fler saker som är intressant. *(förslagsvis från F)*